Alumnos por categoria de buen o mal internet, un grafico para cada componetencia-por grado. 

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from concurrent.futures import ThreadPoolExecutor, as_completed

import athena_utils as athena

In [28]:

def ejecutar_queries_paralelo(queries_dict, max_workers=5):
    """
    Ejecuta múltiples queries de Athena en paralelo.
    
    Args:
        queries_dict (dict): Diccionario donde la clave es el nombre identificador 
                            y el valor es la query SQL
                            Ejemplo: {'cancelaciones': query1, 'asistencias': query2}
        max_workers (int): Número máximo de threads paralelos (default: 5)
    
    Returns:
        dict: Diccionario con los DataFrames resultantes
              Ejemplo: {'cancelaciones': df1, 'asistencias': df2}
    """
    results = {}
    errors = {}
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Enviar todas las queries en paralelo
        futures = {
            executor.submit(athena.run_athena_query_auto, query, name): name 
            for name, query in queries_dict.items()
        }
        
        # Recoger resultados según vayan terminando
        for future in as_completed(futures):
            query_name = futures[future]
            try:
                df = future.result()
                results[query_name] = df
                print(f"✅ {query_name}: {len(df)} filas obtenidas")
            except Exception as e:
                errors[query_name] = str(e)
                print(f"❌ {query_name}: Error - {e}")
    
    # Reportar resumen
    print(f"\n📊 Resumen: {len(results)}/{len(queries_dict)} queries completadas exitosamente")
    
    if errors:
        print(f"⚠️  Queries con error: {list(errors.keys())}")
    
    return results

In [29]:
projects_id= 72

In [ ]:
# Librerías para manejo de documentos Word
from docx import Document
from docx.shared import Inches, Pt, RGBColor
from docx.text.paragraph import Paragraph
from docx.oxml import parse_xml, OxmlElement
from docx.oxml.ns import qn
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT, WD_ALIGN_PARAGRAPH
from docx.enum.table import WD_TABLE_ALIGNMENT, WD_ALIGN_VERTICAL
from docx.enum.style import WD_STYLE_TYPE

# Crear un nuevo documento
doc = Document()
# Configurar página en tamaño A4 (21 x 29.7 cm = 8.27 x 11.69 inches)
section = doc.sections[0]
section.page_height = Inches(11.69)
section.page_width = Inches(8.27)

# Márgenes (por ejemplo, 1 pulgada a cada lado)
section.top_margin = Inches(1)
section.bottom_margin = Inches(1)
section.left_margin = Inches(1)
section.right_margin = Inches(1)


def agregar_titulo(doc, texto, nivel):
    # Paleta de colores corporativos sobrios
    COLOR_TITULO = RGBColor(0x2E, 0x3F, 0x5F)  # Azul marino oscuro
    COLOR_SUBTITULO = RGBColor(0x4F, 0x4F, 0x4F)  # Gris oscuro

    if nivel == 1:
        # Título principal - Nivel 1
        titulo = doc.add_heading(level=1)
        run = titulo.add_run(texto.upper())
        run.font.name = 'Lora'
        run.font.size = Pt(14)
        run.font.bold = True
        run.font.color.rgb = COLOR_TITULO
        titulo.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
        titulo.paragraph_format.space_before = Pt(18)
        titulo.paragraph_format.space_after = Pt(12)
        
        # Agregar línea decorativa inferior
        p = titulo._element
        pPr = p.get_or_add_pPr()
        pBdr = OxmlElement('w:pBdr')
        pPr.append(pBdr)
        bottom = OxmlElement('w:bottom')
        bottom.set(qn('w:val'), 'single')
        bottom.set(qn('w:sz'), '8')
        bottom.set(qn('w:space'), '1')
        bottom.set(qn('w:color'), '2E3F5F')
        pBdr.append(bottom)

    elif nivel == 2:
        # Subtítulo importante - Nivel 2
        titulo = doc.add_heading(level=2)
        run = titulo.add_run(texto)
        run.font.name = 'Lora'
        run.font.size = Pt(12)
        run.font.bold = True
        run.font.color.rgb = COLOR_SUBTITULO
        titulo.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
        titulo.paragraph_format.space_before = Pt(14)
        titulo.paragraph_format.space_after = Pt(8)
        
        # Subrayado decorativo
        p = titulo._element
        pPr = p.get_or_add_pPr()
        pBdr = OxmlElement('w:pBdr')
        pPr.append(pBdr)
        bottom = OxmlElement('w:bottom')
        bottom.set(qn('w:val'), 'single')
        bottom.set(qn('w:sz'), '6')
        bottom.set(qn('w:space'), '1')
        bottom.set(qn('w:color'), 'D3D3D3')
        pBdr.append(bottom)

    elif nivel == 3:
        # Subtítulo secundario - Nivel 3
        titulo = doc.add_heading(level=3)
        run = titulo.add_run(texto)
        run.font.name = 'Lora'
        run.font.size = Pt(11)
        run.font.color.rgb = COLOR_SUBTITULO
        run.font.italic = True
        titulo.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
        titulo.paragraph_format.space_before = Pt(10)
        titulo.paragraph_format.space_after = Pt(4)

    else:
        # Para niveles inferiores
        parrafo = doc.add_paragraph(texto)
        parrafo.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY
        run = parrafo.runs[0]
        run.font.name = 'Segoe UI Light'
        run.font.size = Pt(8)
        run.font.underline = True
        run.font.bold = True
        '''
        titulo = doc.add_heading(level=nivel)
        run = titulo.add_run(texto)
        run.font.name = 'Lora'
        run.font.size = Pt(9)
        run.font.color.rgb = COLOR_SUBTITULO
        titulo.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
        titulo.paragraph_format.space_before = Pt(6)
        titulo.paragraph_format.space_after = Pt(2)
        '''

def agregar_parrafo(doc, texto):
    parrafo = doc.add_paragraph(texto)
    parrafo.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY
    run = parrafo.runs[0]
    run.font.name = 'Segoe UI Light'
    run.font.size = Pt(8)

def insertar_figura(doc, figura, titulo=None, pie=None):
    if titulo:  # Solo agrega título si se proporciona
        agregar_titulo(doc, titulo, 3)
    imagen_stream = BytesIO()
    figura.savefig(imagen_stream, format='png', bbox_inches='tight')
    imagen_stream.seek(0)
    # Insertar imagen centrada
    p = doc.add_paragraph()
    run = p.add_run()
    run.add_picture(imagen_stream, width=Inches(5.5))
    p.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    imagen_stream.close()
    # Insertar pie de gráfico si se proporciona
    if pie:
        pie_p = doc.add_paragraph(pie)
        pie_p.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
        run = pie_p.runs[0]
        run.font.name = 'Segoe UI Light'
        run.font.size = Pt(6)
        run.font.bold = True
        run.font.italic = True

def set_cell_width(cell, width_inches):
    """
    Establece el ancho de una celda en pulgadas.
    """
    width_twips = int(width_inches * 1440)
    cell.width = Inches(width_inches)
    tc = cell._tc
    tcPr = tc.get_or_add_tcPr()
    
    # Eliminar cualquier w:tcW anterior
    for child in tcPr.findall(qn('w:tcW')):
        tcPr.remove(child)

    # Crear nuevo elemento de ancho
    tcW = OxmlElement('w:tcW')
    tcW.set(qn('w:w'), str(width_twips))
    tcW.set(qn('w:type'), 'dxa')
    tcPr.append(tcW)

def insertar_tabla(doc, df, titulo=None):
    if titulo:
        agregar_titulo(doc, titulo, 3)

    tabla = doc.add_table(rows=1, cols=len(df.columns))
    tabla.style = 'Table Grid'
    tabla.alignment = WD_TABLE_ALIGNMENT.CENTER

    ancho_total = 6.0
    ancho_columna = ancho_total / len(df.columns)

    # Encabezados
    hdr_cells = tabla.rows[0].cells
    for i, col_name in enumerate(df.columns):
        cell = hdr_cells[i]
        cell.text = str(col_name)
        run = cell.paragraphs[0].runs[0]
        run.font.bold = True
        run.font.size = Pt(6.5)
        run.font.name = 'Segoe UI Light'
        set_cell_width(cell, ancho_columna)
        # centrar
        cell.paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
        cell.vertical_alignment      = WD_ALIGN_VERTICAL.CENTER

    # Filas de datos
    for _, row in df.iterrows():
        row_cells = tabla.add_row().cells
        for i, value in enumerate(row):
            cell = row_cells[i]
            cell.text = str(value)
            run = cell.paragraphs[0].runs[0]
            run.font.size = Pt(7)
            set_cell_width(cell, ancho_columna)
            # centrar
            cell.paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
            cell.vertical_alignment      = WD_ALIGN_VERTICAL.CENTER

def insertar_tabla_con_merge(doc, df, titulo=None, group_cols=None):
    if titulo:
        agregar_titulo(doc, titulo, 3)

    tabla = doc.add_table(rows=1, cols=len(df.columns))
    tabla.style = 'Table Grid'
    tabla.alignment = WD_TABLE_ALIGNMENT.CENTER

    ancho_total = 6.0
    ancho_columna = ancho_total / len(df.columns)

    # Encabezados
    hdr_cells = tabla.rows[0].cells
    for i, col in enumerate(df.columns):
        cell = hdr_cells[i]
        cell.text = str(col)
        run = cell.paragraphs[0].runs[0]
        run.font.bold = True
        run.font.size = Pt(6.5)
        run.font.name = 'Segoe UI Light'
        # Anchos
        set_cell_width(cell, ancho_columna)
        # Centrado horizontal y vertical
        cell.paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
        cell.vertical_alignment = WD_ALIGN_VERTICAL.CENTER

    # Filas de datos
    for _, row in df.iterrows():
        row_cells = tabla.add_row().cells
        for i, val in enumerate(row):
            cell = row_cells[i]
            cell.text = str(val)
            run = cell.paragraphs[0].runs[0]
            run.font.size = Pt(7)
            set_cell_width(cell, ancho_columna)
            # Centrado horizontal y vertical
            cell.paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
            cell.vertical_alignment = WD_ALIGN_VERTICAL.CENTER

    # Merge de grupos (igual que antes)…
    if group_cols:
        col2idx = {col: idx for idx, col in enumerate(df.columns)}
        sizes = OrderedDict()
        prev_key = None
        for key_vals in df[group_cols].itertuples(index=False, name=None):
            if key_vals == prev_key:
                sizes[key_vals] += 1
            else:
                sizes[key_vals] = 1
                prev_key = key_vals

        current_row = 1
        for key_vals, size in sizes.items():
            if size > 1:
                for col in group_cols:
                    c_idx = col2idx[col]
                    start = tabla.cell(current_row, c_idx)
                    end   = tabla.cell(current_row + size - 1, c_idx)
                    # Vaciar intermedias y merge
                    for r in range(current_row + 1, current_row + size):
                        tabla.cell(r, c_idx).text = ''
                    start.merge(end)
                    # Aseguramos que la celda fusionada mantenga el centrado
                    start.paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
                    start.vertical_alignment = WD_ALIGN_VERTICAL.CENTER
            current_row += size

    return tabla

def insertar_salto_pagina(doc):
    doc.add_page_break()

def agregar_viñetas(doc, items, nivel=1, espacio_antes=Pt(4), espacio_despues=Pt(4)):
    """
    Inserta una lista usando guiones '-' como viñetas.

    Parámetros
    ----------
    doc : Document
        Objeto python-docx Document.
    items : list de str
        Cada cadena será un ítem de la lista.
    nivel : int, opcional (por defecto=1)
        Nivel de sangría (1 = viñetas principales, 2 = sub-viñetas, etc.).
    espacio_antes : Pt, opcional
        Espacio antes de cada ítem.
    espacio_despues : Pt, opcional
        Espacio después de cada ítem.
    """
    indent_por_nivel = Pt(12)  # 12pt de sangría por nivel

    for texto in items:
        # Preparo el párrafo con indentación
        p = doc.add_paragraph()
        p.paragraph_format.space_before = espacio_antes
        p.paragraph_format.space_after = espacio_despues
        # Sangrar según nivel, a la izquierda
        p.paragraph_format.left_indent = indent_por_nivel * (nivel - 1)

        # Agregar el run con guión + texto
        run = p.add_run(f"- {texto}")
        run.font.name = 'Segoe UI Light'
        run.font.size = Pt(8)
        # Alineación por defecto (izquierda)
        p.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT

def insertar_en_posicion(doc, funcion_contenido, *args, posicion='final', **kwargs):
    """
    Inserta contenido generado por una función en una posición específica del documento.

    Parámetros
    ----------
    doc : Document
        Documento principal.
    funcion_contenido : function
        Función que recibe un doc y otros parámetros, y agrega contenido (párrafo, título, etc.).
    *args, **kwargs :
        Argumentos para pasar a la función.
    posicion : str
        'inicio', 'final' o 'index:<n>' para insertar en una posición concreta.
    """
    # Crear documento temporal con el contenido a insertar
    doc_temp = Document()
    funcion_contenido(doc_temp, *args, **kwargs)

    # Extraer elementos del cuerpo
    elementos_temp = list(doc_temp.element.body)

    # Insertar al inicio, final o índice
    body = doc.element.body

    if posicion == 'inicio':
        for elem in reversed(elementos_temp):
            body.insert(0, elem)
    elif posicion == 'final':
        for elem in elementos_temp:
            body.append(elem)
    elif posicion.startswith('index:'):
        idx = int(posicion.split(':')[1])
        for i, elem in enumerate(elementos_temp):
            body.insert(idx + i, elem)
    else:
        raise ValueError("La posición debe ser 'inicio', 'final' o 'index:<n>'")

def insertar_indice(doc, titulo="Índice"):
    # Título del índice
    agregar_titulo(doc, titulo, 1)

    # Párrafo donde irá la tabla de contenido
    p = doc.add_paragraph()
    run = p.add_run()

    # Agregar campo TOC
    fldChar1 = OxmlElement('w:fldChar')
    fldChar1.set(qn('w:fldCharType'), 'begin')

    instrText = OxmlElement('w:instrText')
    instrText.set(qn('xml:space'), 'preserve')
    instrText.text = r'TOC \o "1-3" \h \z \u'

    fldChar2 = OxmlElement('w:fldChar')
    fldChar2.set(qn('w:fldCharType'), 'separate')

    fldChar3 = OxmlElement('w:fldChar')
    fldChar3.set(qn('w:fldCharType'), 'end')

    run._r.append(fldChar1)
    run._r.append(instrText)
    run._r.append(fldChar2)
    run._r.append(fldChar3)

    # Estilo
    p.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
    p.paragraph_format.space_after = Pt(6)

def agregar_advertencia_actualizacion(doc):
    p = doc.add_paragraph()
    run = p.add_run("⚠️ Al abrir este documento, recuerde actualizar los campos (índice, referencias cruzadas, etc.).")
    run.font.italic = True
    run.font.color.rgb = RGBColor(0x80, 0x00, 0x00)
    p.paragraph_format.space_before = Pt(12)

def mostrar_contenido(doc):
    print("Índice | Tipo   | Contenido resumido")
    print("--------------------------------------")

    idx_parrafo = 0
    idx_tabla = 0

    for i, elem in enumerate(doc.element.body):
        tag = elem.tag.split('}')[-1]

        if tag == 'p':
            parrafo = doc.paragraphs[idx_parrafo]
            texto = parrafo.text.strip().replace('\n', ' ')
            print(f"{i:<6} | Párrafo | '{texto[:60]}'")
            idx_parrafo += 1

        elif tag == 'tbl':
            print(f"{i:<6} | Tabla   | [Tabla con {len(doc.tables[idx_tabla].rows)} filas]")
            idx_tabla += 1

        else:
            print(f"{i:<6} | Otro    | Etiqueta: {tag}")
    
def mostrar_contenido_posicional(doc, buscar=None):
    """
    Si se pasa un texto en `buscar`, también devuelve las posiciones donde aparece.
    """
    idx_parrafo = 0
    posiciones_encontradas = []

    for i, elem in enumerate(doc.element.body):
        tag = elem.tag.split('}')[-1]

        if tag == 'p':
            parrafo = doc.paragraphs[idx_parrafo]
            texto = parrafo.text.strip().replace('\n', ' ')
            # print(f"{i:<6} | Párrafo | '{texto[:60]}'")

            if buscar and buscar.lower() in texto.lower():
                posiciones_encontradas.append(i)

            idx_parrafo += 1

    return posiciones_encontradas

def reemplazar_parrafo(original: Paragraph, nuevo: Paragraph):
    # Reemplaza el elemento XML del párrafo original por el del nuevo
    original._element.getparent().replace(original._element, nuevo._element)

def numerar_titulos_existentes(doc):
    contador = {1: 0, 2: 0, 3: 0}
    # Guardar los párrafos a reemplazar (para evitar modificar la lista mientras iteras)
    reemplazos = []

    for i, parrafo in enumerate(doc.paragraphs):
        estilo = parrafo.style.name.strip()
        if estilo.startswith("Heading"):
            try:
                nivel = int(estilo.split()[-1])
            except (ValueError, IndexError):
                continue

            if nivel in contador:
                contador[nivel] += 1
                for deeper in range(nivel + 1, 4):
                    contador[deeper] = 0

                if nivel == 1:
                    numeracion = f"{contador[1]}."
                elif nivel == 2:
                    numeracion = f"{contador[1]}.{contador[2]}"
                elif nivel == 3:
                    numeracion = f"{contador[1]}.{contador[2]}.{contador[3]}"

                texto = parrafo.text.strip()
                if not texto.startswith(numeracion):
                    texto_sin_num = texto
                    # Crear un doc temporal para el nuevo título
                    doc_temp = Document()
                    agregar_titulo(doc_temp, f"{numeracion} {texto_sin_num}", nivel)
                    nuevo_parrafo = doc_temp.paragraphs[0]
                    reemplazos.append((parrafo, nuevo_parrafo))

    # Hacer los reemplazos al final para evitar problemas de índice
    for original, nuevo in reemplazos:
        reemplazar_parrafo(original, nuevo)



In [30]:
query_cancelaciones= f'''

WITH
cte_Organization AS (
    SELECT
        poa.project_id,
        ARRAY_JOIN(ARRAY_AGG(DISTINCT o.name), ', ') AS org_names
    FROM datalake.project_organization_association poa
    JOIN datalake.organizations o ON poa.organization_id = o.id
    GROUP BY poa.project_id
),
cte_ProgramType AS (
    SELECT
        ppta.project_id,
        ARRAY_JOIN(ARRAY_AGG(DISTINCT cbpt.name), ', ') AS program_types
    FROM datalake.project_program_type_association ppta
    JOIN datalake.catalog_b2bprogramtype cbpt ON ppta.program_type_id = cbpt.id
    GROUP BY ppta.project_id
),
base AS (
    SELECT
        DISTINCT
        p.id AS projectsID,
        p.name AS "Proyecto",
        COALESCE(p.type, 'B2C') AS "Canal",
        pt.program_types AS "Tipo de programa",
        o.org_names AS "Organización",
        (CASE WHEN ee.institution IS NULL THEN ei.name ELSE ee.institution END) AS institucion,
        (CASE WHEN ee.group_section IS NULL THEN rr.college_group ELSE ee.group_section END) AS seccion,
        ee.career AS career,
        rr.id AS room,
        rr.name AS "Room Name",
        CONCAT('https://backoffice.crackthecode.la/dashboard/rooms/', CAST(rr.id AS VARCHAR)) AS "Link Room",
        rs.id AS sesionID,
        rs.session_number AS sesion,
        ee.grade AS grado,
        (CASE WHEN rs.cancellation_reason_id IS NULL THEN 36 ELSE rs.cancellation_reason_id END) AS reasonID,
        (CASE WHEN rc.name IS NULL THEN 'N/A' ELSE rc.name END) AS Motivo,
        rs.start_date AS "Fecha",
        rs.start_time AS Hora,
        SUBSTR(rs.start_time, 1, 2) AS "Hora Agrupada",
        DATE_TRUNC('week', rs.start_date) AS ordenSemana,
        CONCAT(
            DATE_FORMAT(DATE_TRUNC('week', rs.start_date), '%d/%m'),
            '-',
            DATE_FORMAT(DATE_TRUNC('week', rs.start_date) + INTERVAL '6' DAY, '%d/%m')
        ) AS semana,
        DATE_FORMAT(rs.start_date, '%W') AS dia,
        rs.state AS state,
        rs.risk_cancellation AS Riesgo,
        CONCAT(au.last_name, ', ', au.first_name) AS profesor
    FROM
        datalake.room_roomsessions rs
        LEFT JOIN datalake.room_room rr ON rs.room_id = rr.id
        JOIN datalake.enrollment_enrolment ee ON rr.id = ee.room_id AND ee.b2b_project_id IS NOT NULL
        LEFT JOIN datalake.projects p ON (ee.b2b_project_id = p.id OR p.id = rr.project_b2b_id)
        LEFT JOIN cte_Organization o ON p.id = o.project_id
        LEFT JOIN cte_ProgramType pt ON p.id = pt.project_id
        LEFT JOIN datalake.account_user au ON rr.teacher_id = au.id
        LEFT JOIN datalake.catalog_reasonsessioncancellation rc ON (
            (CASE WHEN (rs.state = 'false' AND rs.cancellation_reason_id IS NULL) THEN 36 ELSE rs.cancellation_reason_id END) = rc.id
        )
        LEFT JOIN datalake.educational_institution ei ON ei.id = rr.educational_institution_id
    ORDER BY
        DATE_TRUNC('week', rs.start_date) ASC
)
SELECT *
FROM base
WHERE
    NOT (state = 'false' AND projectsID IN (19, 14) AND (("Fecha" BETWEEN DATE '2024-06-17' AND DATE '2024-07-05') OR ("Fecha" BETWEEN DATE '2024-10-07' AND DATE '2024-10-14')))
    AND
    NOT (state = 'false' AND projectsID IN (47, 48, 56) AND ("Fecha" BETWEEN DATE '2024-10-07' AND DATE '2024-10-14'))
    AND
    (reasonID NOT IN (33, 34, 35) OR reasonID IS NULL)
    
    and projectsID IN ({projects_id})
    
    ;


'''

In [31]:
query_asistencias=f'''

SELECT
*
FROM
asistencia_alumnos

where 
b2b_project_id IN ({projects_id})

'''

In [32]:
query_cuestionarios = f''' 

SELECT DISTINCT
     me.moodle_id moodle_user_id
   , 'Moodle' origen
   , ss.id student_id
   , ee.institution educative_institution
   , ee.grade grade
   , concat(ee.grade,'+', ee.group_section) grade_section
   , ee.career career
   , ee.educational_level educational_level
   , DATE_DIFF('year', ss.birthdate, p.operative_start_date) age 
   , ss.gender genero
   , p.id project_id
   , p.name project_name
   , ce.course_id moodle_course_id
   , rr.id room_id
   , ce.unique_id evaluation_unique_id
   , ce.name evaluation_name
   , ceq.name question_name
   , ceq.tag tag_question
   , ceq.question_id question_id
   , ceq.question_name question
   , cer.answer answer
   , cer.right_answer right_answer
   , ce.tag AS tipo_test

   FROM
   moodle_enrollment me
   LEFT JOIN moodle_course_evaluations ce ON (me.course_id = ce.course_id)
   LEFT JOIN moodle_course_evaluation_questions ceq ON (ce.unique_id = ceq.unique_id) AND ((ceq.question_name <> 'label') OR (ceq.question_name IS NULL))
   LEFT JOIN moodle_course_evaluation_responses cer ON ((cer.unique_id = ceq.unique_id) AND (ceq.question_id = cer.question_id) AND (me.moodle_id = cer.moodle_id) AND (ce.type <> 'assign')  AND (cer.attempt_time_finish IS NOT NULL))
   INNER JOIN room_room rr ON (rr.course_mdl_id = me.course_id)
   LEFT JOIN student_student ss ON (ss.user_mdl_id = me.moodle_id)
   LEFT JOIN room_room_students rrs ON ((rrs.student_id = ss.id) AND (rrs.room_id = rr.id))
   LEFT JOIN enrollment_enrolment ee ON (((ee.group_id = rr.group_id) OR (ee.room_id = rr.id)) AND (ee.student_id = ss.id) AND (ee.state <> 'cancel') AND (ee.state <> 'inactive'))
   LEFT JOIN projects p ON (p.id = ee.b2b_project_id)
   
   WHERE (p.id in ({projects_id}) and (me.role = 'student'))



'''

In [33]:
query_alumnos = f'''
select distinct
    p.id as Proyecto,
    ss.id as ID, 
    concat(ss.last_name, ', ', ss.first_name) as Nombre_Completo,
    ss.email as Email,
    ss.phone_number as Telefono,
    ss.doc_type as tipo_documento,
    ss.doc_number as documento,
    ee.institution as Institucion,
    ee.grade as grado,
    ee.group_section as seccion,
    ee.room_id as Salon,
    ss.gender as Genero,
    DATE_DIFF('year', ss.birthdate, p.operative_start_date) as Edad,
    case when ((ee.state <> 'cancel') and (ee.state <> 'inactive')) then 'Activo' else 'Inactivo' end as Status,
	COALESCE(MAX(CASE WHEN ceq.tag = 'estrato_socioeconomico' THEN cer.answer END), 'Sin respuesta') AS estrato_socioeconomico,
	COALESCE(ARRAY_JOIN(ARRAY_AGG(distinct CASE WHEN ceq.tag = 'etnia' THEN cer.answer END) FILTER (WHERE ceq.tag = 'etnia'), '|'), 'Sin respuesta') AS etnia,
    CASE 
        WHEN SUM(CASE WHEN device = 'Mobile' THEN 1 ELSE 0 END) = 0 THEN '1- Solo Desktop'
        WHEN SUM(CASE WHEN device = 'Desktop' THEN 1 ELSE 0 END) = 0 THEN '5- Solo Mobile'
        WHEN (SUM(CASE WHEN device = 'Desktop' THEN 1 ELSE 0 END) / CAST(COUNT(*) AS DOUBLE)) >= 0.8 THEN '2- 80% Desktop'
        WHEN (SUM(CASE WHEN device = 'Mobile' THEN 1 ELSE 0 END) / CAST(COUNT(*) AS DOUBLE)) >= 0.8 THEN '4- 80% Mobile'
        ELSE '3- Mixto'
        END AS dispositivo
	
from
    datalake.enrollment_enrolment ee
    left join datalake.student_student ss on ss.id=ee.student_id
    left join datalake.projects p on p.id=ee.b2b_project_id
	left join datalake.moodle_enrollment me on me.moodle_id=ss.user_mdl_id
	LEFT JOIN datalake.moodle_course_evaluations ce ON (me.course_id = ce.course_id)
   	LEFT JOIN datalake.moodle_course_evaluation_questions ceq ON (ce.unique_id = ceq.unique_id) AND ((ceq.question_name <> 'label') OR (ceq.question_name IS NULL))
   	LEFT JOIN datalake.moodle_course_evaluation_responses cer ON ((cer.unique_id = ceq.unique_id) AND (ceq.question_id = cer.question_id) AND (me.moodle_id = cer.moodle_id) AND (ce.type <> 'assign')  AND (cer.attempt_time_finish IS NOT NULL))
    LEFT JOIN datalake.moodle_session_device msd ON msd.userid=ss.user_mdl_id and msd.ip is not null

where
    p.id in ({projects_id})

group by
	p.id,
    ss.id,
    ss.last_name,
    ss.first_name,
    ss.email,
    ss.phone_number,
    ss.doc_type,
    ss.doc_number,
    ee.institution,
    ee.grade,
    ee.group_section,
    ee.room_id,
    ss.gender,
    ss.birthdate,
    p.operative_start_date,
    ee.state
'''

In [34]:
# Preparar queries
queries = {
    'cancelaciones': query_cancelaciones,
    'asistencias': query_asistencias,
    'cuestionarios': query_cuestionarios,
    'alumnos': query_alumnos
}

# Ejecutar en paralelo
print("🚀 Iniciando ejecución de queries en paralelo...\n")
dataframes = ejecutar_queries_paralelo(queries)

🚀 Iniciando ejecución de queries en paralelo...

✅ cancelaciones: 8972 filas obtenidas
✅ asistencias: 171642 filas obtenidas
✅ alumnos: 8035 filas obtenidas
✅ cuestionarios: 9852429 filas obtenidas

📊 Resumen: 4/4 queries completadas exitosamente


In [35]:
# Acceder a los resultados
df_cancelaciones = dataframes.get('cancelaciones')
df_asistencias = dataframes.get('asistencias')
df_cuestionarios = dataframes.get('cuestionarios')
df_alumnos=dataframes.get('alumnos')

## 📊 Análisis de Distribuciones Demográficas

In [36]:
# 1. DISTRIBUCIÓN POR GÉNERO
dist_genero = df_alumnos['genero'].value_counts().reset_index()
dist_genero.columns = ['Género', 'Cantidad']
dist_genero['Porcentaje'] = (dist_genero['Cantidad'] / dist_genero['Cantidad'].sum() * 100).round(2)

# 2. DISTRIBUCIÓN POR RANGO DE EDAD
def categorizar_edad(edad):
    if pd.isna(edad):
        return 'Sin información'
    elif edad < 12:
        return 'Menos de 12 años'
    elif 12 <= edad <= 14:
        return '12-14 años'
    elif 15 <= edad <= 17:
        return '15-17 años'
    elif 18 <= edad <= 20:
        return '18-20 años'
    else:
        return 'Más de 20 años'

df_alumnos['rango_edad'] = df_alumnos['edad'].apply(categorizar_edad)

dist_edad = df_alumnos['rango_edad'].value_counts().reset_index()
dist_edad.columns = ['Rango de Edad', 'Cantidad']
dist_edad['Porcentaje'] = (dist_edad['Cantidad'] / dist_edad['Cantidad'].sum() * 100).round(2)

# Ordenar por rango
orden_edad = ['Menos de 12 años', '12-14 años', '15-17 años', '18-20 años', 'Más de 20 años', 'Sin información']
dist_edad['Rango de Edad'] = pd.Categorical(dist_edad['Rango de Edad'], categories=orden_edad, ordered=True)
dist_edad = dist_edad.sort_values('Rango de Edad')


# 3. DISTRIBUCIÓN POR ESTRATO SOCIOECONÓMICO
dist_estrato = df_alumnos['estrato_socioeconomico'].value_counts().reset_index()
dist_estrato.columns = ['Estrato Socioeconómico', 'Cantidad']
dist_estrato['Porcentaje'] = (dist_estrato['Cantidad'] / dist_estrato['Cantidad'].sum() * 100).round(2)
dist_estrato = dist_estrato.sort_values('Estrato Socioeconómico')

# 4. DISTRIBUCIÓN POR ETNIA/ASCENDENCIA
# Desagregar etnias concatenadas (separadas por |)
def desagregar_etnias(df, columna_etnia='etnia'):
    """
    Desagrega etnias cuando un estudiante tiene múltiples valores separados por |
    """
    # Lista para almacenar los registros desagregados
    registros_desagregados = []
    
    for idx, row in df.iterrows():
        etnia_valor = row[columna_etnia]
        
        # Si el valor es nulo o vacío
        if pd.isna(etnia_valor) or str(etnia_valor).strip() == '':
            registros_desagregados.append({
                'id': row['id'],
                'etnia_individual': 'Sin información'
            })
        else:
            # Separar por | y crear un registro por cada etnia
            etnias = str(etnia_valor).split(';')
            for etnia in etnias:
                etnia_limpia = etnia.strip()
                if etnia_limpia:  # Solo si no está vacío
                    registros_desagregados.append({
                        'id': row['id'],
                        'etnia_individual': etnia_limpia
                    })
    
    return pd.DataFrame(registros_desagregados)

# Desagregar las etnias
df_etnias_desagregadas = desagregar_etnias(df_alumnos)

# Calcular distribución
dist_etnia = df_etnias_desagregadas['etnia_individual'].value_counts().reset_index()
dist_etnia.columns = ['Etnia/Ascendencia', 'Cantidad de estudiantes']

# Calcular porcentaje sobre el total de estudiantes únicos
total_estudiantes = df_alumnos['id'].nunique()
dist_etnia['Porcentaje (sobre total estudiantes)'] = (dist_etnia['Cantidad de estudiantes'] / total_estudiantes * 100).round(2)

In [37]:
# 1. DISTRIBUCIÓN POR GÉNERO

print("=" * 50)
print("📊 DISTRIBUCIÓN POR GÉNERO")
print("=" * 50)
print(dist_genero.to_string(index=False))
print(f"\nTotal: {dist_genero['Cantidad'].sum()} estudiantes")
print("=" * 50)

📊 DISTRIBUCIÓN POR GÉNERO
     Género  Cantidad  Porcentaje
     female      4791       59.63
       male      3173       39.49
unspecified        71        0.88

Total: 8035 estudiantes


In [38]:
# 2. DISTRIBUCIÓN POR RANGO DE EDAD

print("\n" + "=" * 50)
print("📊 DISTRIBUCIÓN POR RANGO DE EDAD")
print("=" * 50)
print(dist_edad.to_string(index=False))
print(f"\nEdad promedio: {df_alumnos['edad'].mean():.1f} años")
print(f"Edad mínima: {df_alumnos['edad'].min():.0f} años")
print(f"Edad máxima: {df_alumnos['edad'].max():.0f} años")
print("=" * 50)


📊 DISTRIBUCIÓN POR RANGO DE EDAD
   Rango de Edad  Cantidad  Porcentaje
Menos de 12 años         4        0.05
      12-14 años      2796       34.80
      15-17 años      5104       63.52
      18-20 años       125        1.56
  Más de 20 años         6        0.07

Edad promedio: 15.0 años
Edad mínima: 6 años
Edad máxima: 24 años


In [39]:
print("\n" + "=" * 50)
print("📊 DISTRIBUCIÓN POR ESTRATO SOCIOECONÓMICO")
print("=" * 50)
print(dist_estrato.to_string(index=False))
print(f"\nTotal: {dist_estrato['Cantidad'].sum()} estudiantes")
print("=" * 50)


📊 DISTRIBUCIÓN POR ESTRATO SOCIOECONÓMICO
Estrato Socioeconómico  Cantidad  Porcentaje
          1. Estrato 1      2000       24.89
          2. Estrato 2      1883       23.43
          3. Estrato 3      1813       22.56
          4. Estrato 4       459        5.71
          5. Estrato 5        57        0.71
          6. Estrato 6        24        0.30
           7. No lo sé       640        7.97
         Sin respuesta      1159       14.42

Total: 8035 estudiantes


In [40]:
print("\n" + "=" * 70)
print("📊 DISTRIBUCIÓN POR ETNIA/ASCENDENCIA")
print("=" * 70)
print("Nota: Un estudiante puede identificarse con múltiples etnias")
print("-" * 70)
print(dist_etnia.to_string(index=False))
print("-" * 70)
print(f"Total de estudiantes únicos: {total_estudiantes}")
print(f"Total de identificaciones étnicas: {dist_etnia['Cantidad de estudiantes'].sum()}")
print(f"Promedio de etnias por estudiante: {dist_etnia['Cantidad de estudiantes'].sum() / total_estudiantes:.2f}")
print("=" * 70)


📊 DISTRIBUCIÓN POR ETNIA/ASCENDENCIA
Nota: Un estudiante puede identificarse con múltiples etnias
----------------------------------------------------------------------
                           Etnia/Ascendencia  Cantidad de estudiantes  Porcentaje (sobre total estudiantes)
                                  4. Mestizo                     3530                                 44.12
                                   5. Blanco                     1737                                 21.71
                                     6. Otro                     1268                                 15.85
                             Sin información                     1157                                 14.46
2. Negro, afrocolombiano, raizal, palenquero                      638                                  7.97
                                 1. Indígena                      239                                  2.99
                             3. Gitano o ROM                       14     